# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv("output_data/city.csv")
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.6500,115.3333,293.75,38,8,3.59,AU,1636774888
1,1,Contai,21.7786,87.7536,294.90,93,100,2.89,IN,1636774889
2,2,Kaitangata,-46.2817,169.8464,282.56,97,100,5.98,NZ,1636774889
3,3,Esperance,-33.8667,121.9000,291.06,51,100,4.73,AU,1636774889
4,4,Fairbanks,64.8378,-147.7164,256.82,80,90,0.00,US,1636774802


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_df[["Lat", "Lng"]].astype(float)
weight = city_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
perfect_df = city_df.loc[(city_df["Cloudiness"] == 0) & (city_df["Wind Speed"] < 2)]
perfect_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
90,90,Vicuña,-30.0319,-70.7081,282.00,59,0,0.18,CL,1636774921
98,98,Salalah,17.0151,54.0924,296.20,49,0,1.54,OM,1636774924
99,99,Jiuquan,39.7432,98.5174,276.56,20,0,1.73,CN,1636774925
191,191,Faya,18.3851,42.4509,282.44,55,0,1.72,SA,1636774961
202,202,Abhā,18.2164,42.5053,282.29,50,0,0.00,SA,1636774965
261,261,Ürümqi,43.8010,87.6005,270.68,59,0,1.00,CN,1636774923
273,273,Coquimbo,-29.9533,-71.3436,286.00,87,0,1.03,CL,1636774993
288,288,Río Gallegos,-51.6226,-69.2181,282.18,61,0,1.54,AR,1636774999
345,345,Ocós,14.5094,-92.1933,300.55,73,0,1.16,GT,1636775021
436,436,Amasya,40.6533,35.8331,272.00,57,0,0.89,TR,1636775058


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
target_coordinates = perfect_df[["Lat", "Lng"]]
target_radius = 8000
target_type = "lodging"
lat_list = []
lng_list = []
name = []

for index, coord in target_coordinates.iterrows():
    lat = coord['Lat']
    lng = coord['Lng']
    target_radius = 5000
    target_type = "lodging"

    payload = {
        "location": f"{lat}, {lng}",
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=payload).json()
    try:
        lat_list.append(response['results'][0]['geometry']['location']['lat'])
        lng_list.append(response['results'][0]['geometry']['location']['lng'])
        name.append(response['results'][0]['name'])
    except:
        print("Could not find coords...")

Could not find coords...


In [7]:
city = ['Vicuña', 'Salalah', 'Jiuquan', 'Faya', 'Abhā', 'Ürümqi', 'Coquimbo', 'Río Gallegos', 'Ocós', 'Amasya', 'Ginda', 'Damaturu', 'Kimberley']
country = ['CL', 'OM', 'CN', 'SA', 'SA', 'CN', 'CL', 'AR', 'GT', 'TR', 'IN', 'NG', 'ZA']

hotel_df = pd.DataFrame({"Hotel Name": name,
                         "City": city,
                         "Country": country,
                        "Lat": lat_list,
                        "Lng": lng_list})
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,Los Mosaicos,Vicuña,CL,-30.032753,-70.711912
1,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel",Salalah,OM,17.019113,54.061124
2,Jiuquan Hotel,Jiuquan,CN,39.737397,98.507802
3,املاك خالد الشعشوع,Faya,SA,18.369083,42.470002
4,Abha Hotel,Abhā,SA,18.213324,42.499891
5,Hua Ling Grand Hotel Urumqi,Ürümqi,CN,43.795200,87.580200
6,Punto de encuentro,Coquimbo,CL,-29.950932,-71.338054
7,Hotel Santa Cruz,Río Gallegos,AR,-51.622810,-69.212223
8,Auto Hotel El Soñador,Ocós,GT,14.508698,-92.193460
9,Lalehan Hotel,Amasya,TR,40.657091,35.835441


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[['Lat', 'Lng']]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
                             
# Display figure
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))